# Summary

The following is the step taken to clean the datasets
- Removing duplicates based on 'raw_mail' column
- Change the "None" to an actuall None value to all column
- Remove email address that is not in the same format
- Fill empty/none email address with bfill and ffill
- Fill empty/none subject with bffill and ffill
- Update the 'date' so it is in one format
- Add malicious column

# import libraries

In [33]:
import pandas as pd
from datetime import datetime
from dateutil import parser
import re

Parse the datasets into dataframe

In [34]:
fraudDataframe = pd.read_json('datasets/raw/fradulent_emails.json', orient='index')
phishingDataframe = pd.read_json('datasets/raw/phishing-chorpus.json', orient='index')
enronDataframe = pd.read_csv('datasets/raw/enron-emails.csv')

Remove duplicates

In [35]:
fraudDataframe = fraudDataframe.drop_duplicates(subset="raw_mail")
phishingDataframe = phishingDataframe.drop_duplicates(subset="raw_mail")
enronDataframe = enronDataframe.drop_duplicates(subset="raw_mail")

In [36]:
print(fraudDataframe.info())
fraudDataframe.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3939 entries, 0 to 3977
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   raw_mail  3939 non-null   object
 1   subject   3939 non-null   object
 2   from      3939 non-null   object
 3   to        3939 non-null   object
 4   status    3939 non-null   object
 5   date      3939 non-null   object
 6   body      3939 non-null   object
dtypes: object(7)
memory usage: 246.2+ KB
None


,raw_mail,subject,from,to,status,date,body
1494,Return-Path: <mlipenza@hotmail.com>\nX-Sieve: ...,CONFIDENTIAL,"""Micheal Ipenza"" <mlipenza@hotmail.com>",R@S,O,"Fri, 11 Mar 2005 17:20:12 -0800",Dear PRESIDENT=2FDIRECTOR=2C \n\nMy name is Mr...
3063,Return-Path: <aj_suleiman@hotmail.com>\nX-Siev...,COMPENSATION FOR ALL THE PAST EFFORT,"""suleiman ahmed"" <aj_suleiman@hotmail.com>",undisclosed-recipients: ;,RO,"Fri, 22 Sep 2006 20:01:36 +0000",<html><div style='background-color:'><DIV>\n<D...
150,Return-Path: <danielkalukalu@netscape.net>\n\t...,GOOD DAY,"""MR DANIEL KALU"" <danielkalukalu@netscape.net>",R@M,O,"Sat, 04 Oct 2003 06:17:14",danielkalukalu@netscape.net\nkalukalu2003@ecpl...
2707,Return-Path: <aishamohm@she.com>\nX-Sieve: CMU...,CONFIDENTIAL PROPOSAL FROM AISHA MOHAMMED,"""MRS AISHA MOHAMMED"" <aishamohm@she.com>",R@M,O,"Mon, 19 Jun 2006 16:39:45 +0700",From=3AMrs MOHAMMED=2E \nAttn=3AThe Managing D...
3053,"X-Sieve: CMU Sieve 2.3\n\tWed, 20 Sep 2006 00:...",PAYMENT REPRESENTATIVE OFFER,"""Dr. Thomas D. Owen Jr"" <spresearch25@myway.co...",None,O,"Wed, 20 Sep 2006 00:03:53 -0400","Dear Sir/Madam,\n\nHow are you doing. I came a..."


In [37]:
print(phishingDataframe.info())
phishingDataframe.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4190 entries, 0 to 4195
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   raw_mail  4190 non-null   object
 1   subject   4190 non-null   object
 2   from      4190 non-null   object
 3   to        4190 non-null   object
 4   status    4190 non-null   object
 5   date      4190 non-null   object
 6   body      4190 non-null   object
dtypes: object(7)
memory usage: 261.9+ KB
None


,raw_mail,subject,from,to,status,date,body
2566,Return-Path: <support@chaseonline.chase.com>\n...,Notification from Online Customer Support Depa...,"""Chase Bank Inc."" <support@chaseonline.chase.com>","user@example.com, nelson@example.com, slashdot...",O,"Thu, 01 Feb 2007 15:53:39 -0200","<html><head><style type=3D""text/css""><!--\r\nb..."
3145,Return-Path: <Paypal@paypal.com>\nX-Original-T...,Regular Maintenance - Review Your Account Details,"""Paypal"" <Paypal@paypal.com>",PayPal@tvconline.net,RO,"Mon, 21 May 2007 16:55:52 -0400",<HTML><HEAD><TITLE></TITLE>\r\n</HEAD>\r\n<BOD...
3511,Return-Path: <user@mail.example.com>\nDelivere...,Security Update,"""Bank Of The West"" <confirm@bankofthewest.com>",user@biochemistry.bioc.cwru.edu,RO,"Mon, 13 Jun 2005 09:52:58 +0100","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
1628,Return-Path: <jimts@earthlink.net>\nX-Original...,Notification,Update Now <jimts@earthlink.net>,undisclosed-recipients: ;,O,"Wed, 8 Mar 2006 17:19:08 -0500",<html>\r\n<head>\r\n<title></title>\r\n<meta h...
1805,Return-Path: <chaseonline@chaseonline.chase.co...,New Message from Chase Online(SM),"""Chase Online Banking"" <chaseonline@chaseonlin...","nobody@example.com, fern@example.com",RO,"Wed, 29 Mar 2006 23:30:29 +0100",<html>\r\n<head>\r\n<!--eBay V3- msxml 4.0 XXX...


In [38]:
print(enronDataframe.info())
enronDataframe.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517401 entries, 0 to 517400
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  517401 non-null  int64  
 1   body        517401 non-null  object 
 2   subject     498214 non-null  object 
 3   raw_mail    517401 non-null  object 
 4   from        517401 non-null  object 
 5   to          495554 non-null  object 
 6   status      0 non-null       float64
 7   date        517401 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 35.5+ MB
None


,Unnamed: 0,body,subject,raw_mail,from,to,status,date
424495,424495,"Attached, in Word format, are (hopefully) the ...",Hawaii I Transaction Documents (Word),Message-ID: <9050907.1075844470536.JavaMail.ev...,murielmcfarling@akllp.com,"alan.quaintance@enron.com, angela.davis@enron....",NaN,"Thu, 16 Nov 2000 17:00:00 -0800 (PST)"
406659,406659,Deal #Q75558.2 - Counterparty is CIA Papelera ...,New Counterparty,Message-ID: <6611728.1075846778509.JavaMail.ev...,susan.scott@enron.com,gcp_houston@enron.com,NaN,"Thu, 2 Nov 2000 07:57:00 -0800 (PST)"
344968,344968,Hi Romeo - \n See Gerald's comments below rega...,Web Site Format,Message-ID: <30279686.1075842621121.JavaMail.e...,michael.eiben@enron.com,romeo.d'souza@enron.com,NaN,"Mon, 23 Oct 2000 02:49:00 -0700 (PDT)"
175066,175066,This is looking better all the time. Looks lik...,Re: Conference call change,Message-ID: <2750909.1075860886186.JavaMail.ev...,rich.jolly@enron.com,bob.burleson@enron.com,NaN,"Mon, 29 Jan 2001 14:03:00 -0800 (PST)"
213317,213317,"Please note that as of January 23, 2002 Reuter...",#FACS/DATA REUTERS BENCHMARK BONDS AS OF Jan 2...,Message-ID: <2354602.1075840749834.JavaMail.ev...,dbnotes@dri-wefa.com,"sinisi./o=dri-wefa/ou=first@enron.com, e-mail ...",NaN,"Mon, 28 Jan 2002 05:31:03 -0800 (PST)"


From a quick glance from all the 3 datasets, there are multiple inconsistensy that can be found in the format of the values. 

- from and to columns contains not only the emails
- datetime isn't in one format

In [39]:
print(fraudDataframe.isna().sum(), '\n') # contains none but inst register as one
print(phishingDataframe.isna().sum(), '\n') # contains none but isnt register as one
print(enronDataframe.isna().sum()) # contains null values

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64 

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64 

Unnamed: 0         0
body               0
subject        19187
raw_mail           0
from               0
to             21847
status        517401
date               0
dtype: int64


As the results show above only the enron sets register having a null values while in fact all 3 datasets does contains a null values

If we check the values for a "None" in a string format we will infact found that the rest of the datasets does in fact contains a Null value 

In [40]:
print((fraudDataframe == "None").sum())
print((phishingDataframe == "None").sum())
print((enronDataframe == "None").sum())

raw_mail      0
subject      17
from        365
to          948
status        0
date        534
body          0
dtype: int64
raw_mail     0
subject     49
from         4
to           9
status       5
date         3
body         0
dtype: int64
Unnamed: 0    0
body          0
subject       0
raw_mail      0
from          0
to            0
status        0
date          0
dtype: int64


Update the datasets to change the "None" values to an actual None

In [41]:
def updateToNone(val):
    if val == "None":
        return None
    else:
        return val

fraudDataframe['subject'] = fraudDataframe['subject'].apply(updateToNone)
fraudDataframe['to'] = fraudDataframe['to'].apply(updateToNone)
fraudDataframe['from'] = fraudDataframe['to'].apply(updateToNone)
fraudDataframe['status'] = fraudDataframe['status'].apply(updateToNone)
fraudDataframe['date'] = fraudDataframe['date'].apply(updateToNone)

print((fraudDataframe == "None").sum())
print(fraudDataframe.isna().sum()) 

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64
raw_mail      0
subject      17
from        948
to          948
status        0
date        534
body          0
dtype: int64


In [42]:
phishingDataframe['subject'] = phishingDataframe['subject'].apply(updateToNone)
phishingDataframe['to'] = phishingDataframe['to'].apply(updateToNone)
phishingDataframe['from'] = phishingDataframe['to'].apply(updateToNone)
phishingDataframe['status'] = phishingDataframe['status'].apply(updateToNone)
phishingDataframe['date'] = phishingDataframe['date'].apply(updateToNone)

print((fraudDataframe == "None").sum())
print(fraudDataframe.isna().sum()) 

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64
raw_mail      0
subject      17
from        948
to          948
status        0
date        534
body          0
dtype: int64


To fix the inconsistensy format of the email in columns "from" and "to" , we will used regex to extract the valid emails first then, fill the empty values with valid values from the datasets

In [43]:
notValidEmail = phishingDataframe['from'].str.contains('[a-zA-Z0-9-_.]*@a-zA-Z0-9-]*(\.[a-zA-Z]*)*', regex=True) == False
phishingDataframe[notValidEmail]['from'].count()

C:\Users\candi\AppData\Local\Temp\ipykernel_3108\3334997152.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  notValidEmail = phishingDataframe['from'].str.contains('[a-zA-Z0-9-_.]*@a-zA-Z0-9-]*(\.[a-zA-Z]*)*', regex=True) == False


4181

In [44]:
notValidEmail = fraudDataframe['from'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False
fraudDataframe[notValidEmail]['from'].count()

C:\Users\candi\AppData\Local\Temp\ipykernel_3108\1799348055.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  notValidEmail = fraudDataframe['from'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False


545

In [45]:
notValidEmail = enronDataframe['to'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False
enronDataframe[notValidEmail]['to'].count()

C:\Users\candi\AppData\Local\Temp\ipykernel_3108\2401131589.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  notValidEmail = enronDataframe['to'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False


16

In [46]:
parsedFrom = fraudDataframe['from'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')
parsedTo = fraudDataframe['to'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')

fraudDataframe['parsed_from'] = parsedFrom[0]
fraudDataframe['parsed_to'] = parsedTo[0]

In [47]:
parsedFrom = phishingDataframe['from'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')
parsedTo = phishingDataframe['to'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')

phishingDataframe['parsed_from'] = parsedFrom[0]
phishingDataframe['parsed_to'] = parsedTo[0]

In [48]:
parsedFrom = enronDataframe['from'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')
parsedTo = enronDataframe['to'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')

enronDataframe['parsed_from'] = parsedFrom[0]
enronDataframe['parsed_to'] = parsedTo[0]

Bellow is the kind of fields that wasnt register as an email

In [49]:
print(fraudDataframe[fraudDataframe['parsed_from'].isna()]['from'].unique())
print(fraudDataframe[fraudDataframe['parsed_to'].isna()]['to'].unique())
print(phishingDataframe[phishingDataframe['parsed_from'].isna()]['from'].unique())
print(phishingDataframe[phishingDataframe['parsed_to'].isna()]['to'].unique())

[None 'undisclosed-recipients: ;' 'undisclosed-recipients:;' ''
 'undisclosed recipients: ;' 'N/A <>, N/A <>' 'N/A <>']
[None 'undisclosed-recipients: ;' 'undisclosed-recipients:;' ''
 'undisclosed recipients: ;' 'N/A <>, N/A <>' 'N/A <>']
['undisclosed-recipients: ;' '[removed]' None 'undisclosed-recipients:;'
 'unlisted-recipients:; (no To-header on input)'
 '<Undisclosed-Recipient:;>' '=?euc-kr?B?u+e2+7nnu/W6rsbtwfawocG3?=' '']
['undisclosed-recipients: ;' '[removed]' None 'undisclosed-recipients:;'
 'unlisted-recipients:; (no To-header on input)'
 '<Undisclosed-Recipient:;>' '=?euc-kr?B?u+e2+7nnu/W6rsbtwfawocG3?=' '']


In [50]:
phishingDataframe['parsed_from'] = phishingDataframe['parsed_from'].ffill().bfill()
fraudDataframe['parsed_from'] = fraudDataframe['parsed_from'].ffill().bfill()
enronDataframe['parsed_from'] = enronDataframe['parsed_from'].ffill().bfill()

In [51]:
phishingDataframe['parsed_to'] = phishingDataframe['parsed_to'].ffill().bfill()
fraudDataframe['parsed_to'] = fraudDataframe['parsed_to'].ffill().bfill()
enronDataframe['parsed_to'] = enronDataframe['parsed_to'].ffill().bfill()

all the datasets subject column contains a null value, we will fill this value using existing fields in the datasets

In [52]:
phishingDataframe['subject'] = phishingDataframe.subject.ffill().bfill()
fraudDataframe['subject'] = fraudDataframe.subject.ffill().bfill()
enronDataframe['subject'] = enronDataframe.subject.ffill().bfill()

In [53]:
print('Number of row that have empty subject for phishingDataframe:', phishingDataframe.subject.isnull().sum())
print('Number of row that have empty subject for fraudDataframe:', fraudDataframe.subject.isnull().sum())
print('Number of row that have empty subject for enronDataframe:', enronDataframe.subject.isnull().sum())

Number of row that have empty subject for phishingDataframe: 0
Number of row that have empty subject for fraudDataframe: 0
Number of row that have empty subject for enronDataframe: 0


In [54]:
enronDataframe['parsed_date'] = enronDataframe.date.apply(lambda date: parser.parse(date).isoformat())

Fill empty fields so no null exist by doing backward and forward fill

In [55]:
fraudDataframe.date = fraudDataframe.date.ffill().bfill()
phishingDataframe.date = phishingDataframe.date.ffill().bfill()

In [56]:
diff = phishingDataframe.shape[0] - phishingDataframe.date.str.contains('[A-Za-z]{0,3}, \d* [A-Za-z]{0,3} \d{4}').sum()
print("Total date row that are not in format for phishingDataframe:", diff)
diff = fraudDataframe.shape[0] - fraudDataframe.date.str.contains('[A-Za-z]{0,3}, \d* [A-Za-z]{0,3} \d{4}').sum()
print("Total date row that are not in format for phishingDataframe:", diff)

Total date row that are not in format for phishingDataframe: 444
Total date row that are not in format for phishingDataframe: 151


In [57]:
def parseDate(date):
    try:
        return parser.parse(date).isoformat()
    except Exception as e:
        return None

In [58]:
phishingDataframe['parsed_date'] = phishingDataframe.date.str.replace('\.', ':', regex=True)
phishingDataframe['parsed_date'] = phishingDataframe['parsed_date'].apply(parseDate)


c:\Users\candi\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Manual cleaning for cases that are to few to automate

In [59]:
phishingDataframe.loc[821].parsed_date = parser.parse("Fri, 09 Jun 2006 08:23:29 +0500 (EST)").isoformat()
phishingDataframe.loc[892].parsed_date = parser.parse("Fri, 23 Jun 2006 13:25:46 -0100 (EST)").isoformat()
phishingDataframe.loc[896].parsed_date = parser.parse("Fri, 23 Jun 2006 21:36:05 +0800").isoformat()
phishingDataframe.loc[1066].parsed_date = parser.parse("Wed, 26 Jul 2006 09:48:28 -0800").isoformat()
phishingDataframe.loc[1067].parsed_date = parser.parse("Wed, 26 Jul 2006 12:50:48 -0600").isoformat()
phishingDataframe.loc[1072].parsed_date = parser.parse("Thu, 27 Jul 2006 03:06:10 -0800").isoformat()
phishingDataframe.loc[1074].parsed_date = parser.parse("Wed, 26 Jul 2006 15:24:52 -0500").isoformat()
phishingDataframe.loc[1075].parsed_date = parser.parse("Wed, 26 Jul 2006 15:43:42 -0500").isoformat()
phishingDataframe.loc[1076].parsed_date = parser.parse("Wed, 26 Jul 2006 19:03:49 -0300").isoformat()
phishingDataframe.loc[1077].parsed_date = parser.parse("Wed, 26 Jul 2006 19:35:02 -0300").isoformat()
phishingDataframe.loc[1095].parsed_date = parser.parse("31.07.2006").isoformat()
phishingDataframe.loc[1173].parsed_date = parser.parse("Thu, 3 Aug 2006 00:13:00 -0530").isoformat()
phishingDataframe.loc[2421].parsed_date = parser.parse("Tue, 09 Jan 2007 14:00:44 +0430").isoformat()
phishingDataframe.loc[3540].parsed_date = parser.parse("Sun, 10 Sep 2006 14:00:47 +0000").isoformat()
phishingDataframe.loc[3643].parsed_date = parser.parse("Fri, 09 Mar 2007 18:11:57 +0530").isoformat()
phishingDataframe.loc[3896].parsed_date = parser.parse("07.08.2006").isoformat()
phishingDataframe.loc[3963].parsed_date = parser.parse("Mon, 24 Feb 2003 17:32:08 +0000").isoformat()
phishingDataframe.loc[4117].parsed_date = parser.parse("Sun, 10 Sep 2006 12:08:54 -0300").isoformat()

In [60]:
fraudDataframe['parsed_date'] = fraudDataframe.date.str.replace('\.', ':', regex=True)
fraudDataframe['parsed_date'] = fraudDataframe['parsed_date'].apply(parseDate)

c:\Users\candi\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname BST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [61]:
def myfunc(row):
    if row.parsed_date == None:
        try:
            row.parsed_date = parser.parse(
                re.search("([A-Za-z]{1,3}, \d{0,2} [A-Za-z]* \d{2,4} \d{2}:\d{2}:\d{2} ((\+|\-)?\d{4})?)", 
                          row.date).group(1)).isoformat()
            return row
        except Exception as e:
            return row
    else:
        return row

fraudDataframe = fraudDataframe.apply(myfunc, axis=1)

Manual Updates

In [62]:
fraudDataframe.loc[542].parsed_date = parser.parse("Sun, 09 nov 2003 21:18:28").isoformat()
fraudDataframe.loc[1236].parsed_date = parser.parse("Tue, 09 nov 2004 15:38:35 -0300").isoformat()

Add prediction label

In [63]:
fraudDataframe['malicious'] = True
phishingDataframe['malicious'] = True
enronDataframe['malicious'] = False

In [64]:
fraudDataframe.to_csv(path_or_buf='datasets/clean/fraud-emails.csv', index=False)
phishingDataframe.to_csv(path_or_buf='datasets/clean/phishing-emails.csv', index=False)
enronDataframe.to_csv(path_or_buf='datasets/clean/enron-emails.csv', index=False)